## Installation

In [ ]:
!pip install ranx

### Imports

In [ ]:
import os
import gzip
import logging
from collections import defaultdict
from ranx import Qrels, Run, fuse, optimize_fusion, evaluate
from google.colab import drive
from ranx import fuse, optimize_fusion, evaluate
import numpy as np
from google.colab import drive
import os
from ranx import Run

## Some Preparation

### Get qrels

In [ ]:
data_folder = 'trec2019-data'
os.makedirs(data_folder, exist_ok=True)

!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz
!tar -xvzf queries.tar.gz

model_save_path = "/content/gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-distilbert-distilroberta-base-2024-05-12_07-39-41"

queries = {}
queries_filepath = os.path.join(data_folder, 'msmarco-test2019-queries.tsv.gz')
if not os.path.exists(queries_filepath):
    logging.info("Download " + os.path.basename(queries_filepath))
    !wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz -O {queries_filepath}

with gzip.open(queries_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

relevant_docs = defaultdict(lambda: defaultdict(int))
qrels_filepath = os.path.join(data_folder, '2019qrels-pass.txt')

if not os.path.exists(qrels_filepath):
    logging.info("Download " + os.path.basename(qrels_filepath))
    !wget https://trec.nist.gov/data/deep/2019qrels-pass.txt -O {qrels_filepath}

with open(qrels_filepath) as fIn:
    for line in fIn:
        qid, _, pid, score = line.strip().split()
        score = int(score)
        if score > 0:
            relevant_docs[qid][pid] = score

relevant_qid = [qid for qid in queries if len(relevant_docs[qid]) > 0]

qrels = Qrels(relevant_docs)

### Get Run Files

In [ ]:

drive.mount('/content/gdrive')

base_path = "/content/gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/"

file_paths = [
    "finetuned_models/cross-encoder-cross-encoder-ms-marco-MiniLM-L-2-v2-2024-05-10_20-46-58ranking.run",
    "finetuned_models/cross-encoder-cross-encoder-ms-marco-TinyBERT-L-2-v2-2024-05-11_07-11-32ranking.run",
    "finetuned_models/cross-encoder-distilbert-distilroberta-base-2024-05-12_07-39-41ranking.run"
]

runs = {}
runs['mini_tiny'] = []
runs['mini_dist'] = []
runs['tiny_dist'] = []
for file_path in file_paths:
    full_path = os.path.join(base_path, file_path)
    run = Run.from_file(full_path, kind="trec")
    # runs.append(run)
    if 'MiniLM' in file_path:
        runs['mini_tiny'].append(run)
        runs['mini_dist'].append(run)
    elif 'distilroberta' in file_path:
        runs['mini_dist'].append(run)
        runs['tiny_dist'].append(run)
    else:
        runs['mini_tiny'].append(run)
        runs['tiny_dist'].append(run)

common_qids = set(qrels.qrels.keys()).intersection(*[set(run.keys()) for run_list in runs.values() for run in run_list])

filtered_qrels = {qid: {doc_id: score for doc_id, score in qrels[qid].items()} for qid in common_qids}
filtered_runs = {key: [{qid: run[qid] for qid in common_qids} for run in run_list] for key, run_list in runs.items()}

qrels = Qrels(filtered_qrels)
runs = {key: [Run(run) for run in run_list] for key, run_list in filtered_runs.items()}

### Get best_params

## Ensemble Methods

### 1. RRF (Reciprocal Rank Fusion, Rank-based Method)

In [ ]:
# 1. RRF (Reciprocal Rank Fusion, Rank-based Method)
for key, run_list in runs.items():
    print(key,":")

    best_params = optimize_fusion(
        qrels=qrels,
        runs=run_list,
        norm="min-max",
        method="rrf",
        metric="ndcg@100"
    )

    combined_run = fuse(
        runs=run_list,
        norm="min-max",
        method="rrf",
        params=best_params,
    )

    metrics = evaluate(
        qrels=qrels,
        run=combined_run,
        metrics=["ndcg@10", "recall@100", "map@1000"]
    )

    print(metrics)

    ndcg_10 = metrics['ndcg@10'] * 100
    recall_100 = metrics['recall@100'] * 100
    map_1000 = metrics['map@1000'] * 100

    print(f"Queries: {len(common_qids)}")
    print(f"NDCG@10: {ndcg_10:.2f}")
    print(f"Recall@100: {recall_100:.2f}")
    print(f"MAP@1000: {map_1000:.2f}")

### 2. BayesFuse

In [ ]:
# 2. BayesFuse
for key, run_list in runs.items():
    print(key,":")

    best_params = optimize_fusion(
        qrels=qrels,
        runs=run_list,
        norm="min-max",
        method="bayesfuse",
        metric="ndcg@100"
    )
    combined_run = fuse(
        runs=run_list,
        norm="min-max",
        method="bayesfuse",
        params=best_params,
    )

    metrics = evaluate(
        qrels=qrels,
        run=combined_run,
        metrics=["ndcg@10", "recall@100", "map@1000"]
    )

    print(metrics)

    ndcg_10 = metrics['ndcg@10'] * 100
    recall_100 = metrics['recall@100'] * 100
    map_1000 = metrics['map@1000'] * 100

    print(f"Queries: {len(common_qids)}")
    print(f"NDCG@10: {ndcg_10:.2f}")
    print(f"Recall@100: {recall_100:.2f}")
    print(f"MAP@1000: {map_1000:.2f}")